In [3]:
import geopandas as gpd
from tqdm import tqdm
import libpysal as lp
import json
import pandas as pd

In [4]:
def retrieve_data(sy):
    """
    Retrieve data for a given school year (sy)
    """
    data_dir = "./LCPS_data"
    # Read the data files
    schools = gpd.read_file('{}/LCPS_Sites_{}.shp'.format(data_dir, sy))
    students = gpd.read_file('{}/Students_{}.shp'.format(data_dir, sy))
    #spas = gpd.read_file('{}/PlanningZones_{}.shp'.format(data_dir, sy))
    
    return students, schools


In [5]:
sy = '2017_2018'

In [6]:
students, schools = retrieve_data(sy)

# Comparing student population to capacity (WRONG APPROACH)

In [7]:
schools.head()


,OBJECTID,SCH_CODE,CLASS,SCH_NUM,NAME,DATE_OPENE,BUILDING_D,CLASSROOMS,SPECIAL_SI,TRAILERS,...,STRT_GRD,END_GRD,ELEM_,INT_,MID_,HIGH_,CAPACITY,PERM_CLRM,PORT_CLRM,geometry
0,1,HCA,CHARTER,119,HILLSBORO CHARTER ACADEMY,1966,7,6,0,0,...,-1,5,119,0,0,0,0,6,0,POINT (11704031.792 7120651.489)
1,2,MSE,ELEMENTARY,155,MOOREFIELD STATION ES,2013,46,48,0,0,...,-1,5,155,0,0,0,1003,48,0,POINT (11765618.437 7048564.330)
2,3,CTY,ELEMENTARY,111,COUNTRYSIDE ES,2003,39,40,0,0,...,-1,5,111,0,0,0,815,40,0,POINT (11791339.593 7064856.835)
3,4,HUT,ELEMENTARY,122,HUTCHISON FARM ES,2002,37,38,2,0,...,-1,5,122,0,0,0,862,38,0,POINT (11761074.023 7017323.603)
4,5,MTC,HIGH,312,CS MONROE TECHNOLOGY,1977,0,0,0,0,...,9,12,0,0,0,312,0,0,0,POINT (11743154.119 7090534.950)


In [8]:
#schools['SCH_NUM']
#schools['CAPACITY']
# imbalance = schools['SCH_NUM'] > schools['CAPACITY']
# num_imbalance = sum(imbalance)
# print('The number of imbalanced/ overpopulated schools is', num_imbalance)
# balance = len(schools['CAPACITY']) - num_imbalance
# print('The number of balanced schools is', balance)

In [9]:
# schools[imbalance]
# charter_schools = schools['CLASS']=='CHARTER'
# schools[charter_schools]

#interested in the charter schools since its capacity is 0
#printed out the schoools that were imbalanced since that gave clarity on the overpopulated schools

In [10]:
# schools_elem_code = schools[['OBJECTID', 'SCH_CODE']]
# schools_elem_code.groupby('SCH_CODE').agg('count')
schools.keys()

Index(['OBJECTID', 'SCH_CODE', 'CLASS', 'SCH_NUM', 'NAME', 'DATE_OPENE',
       'BUILDING_D', 'CLASSROOMS', 'SPECIAL_SI', 'TRAILERS', 'SCHL_CODE',
       'STRT_GRD', 'END_GRD', 'ELEM_', 'INT_', 'MID_', 'HIGH_', 'CAPACITY',
       'PERM_CLRM', 'PORT_CLRM', 'geometry'],
      dtype='object')

# To compare student population to capacity (within a district) 

In [11]:
students.keys()
# students['GRADE']


Index(['ObjectID', 'ARC_Single', 'Loud_ID', 'Address', 'Address_Ex', 'ADDwEXT',
       'Prefix', 'Street_Nam', 'Street_Typ', 'Suffix', 'Town', 'Zip_1',
       'Subdivisio', 'GRID_CODE', 'GRADE', 'IEP_FLAG', 'GENDER', 'ETHNIC',
       'BIRTH', 'APT_PH', 'STATE_PH', 'LEP_FLAG', 'FSI', 'Current_S',
       'Previous_S', 'ELEM_CODE', 'INT_CODE', 'HIGH_CODE', 'geometry'],
      dtype='object')

### Population from Students dataframe

In [12]:
# students.head()
#I'm interested in ELEM_code but I needed another column for the count so ObjectID
#was chosen
#Direct groupby with student district(ELEM_CODE)
elementary_students = students[students['GRADE'] <= 5]
students_elem_code = elementary_students[['ObjectID', 'ELEM_CODE']]
students_elem_code
grouped_df = students_elem_code.groupby(['ELEM_CODE']).agg('count')
grouped_df.rename(columns={"ObjectID":"Num_Students"}, inplace=True)
grouped_df.head()



,Num_Students
ELEM_CODE,
ALD,162
ALG,395
ARC,829
ASH,517
BAL,534


### Capacity from Schools dataframe

In [13]:
# schools['CAPACITY']
elementary_schools = schools[schools['CLASS'] == 'ELEMENTARY']
output_df2 = elementary_schools[['SCH_CODE', 'CAPACITY']]
output_df2.rename(columns={"SCH_CODE":"ELEM_CODE"}, inplace=True)
output_df2.head()

C:\Users\james\Anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,ELEM_CODE,CAPACITY
1,MSE,1003
2,CTY,815
3,HUT,862
5,BUF,1039
6,LIN,138


### Merged dataframes

In [14]:
merged_df = pd.merge(grouped_df, output_df2, on='ELEM_CODE')
merged_df.head()

,ELEM_CODE,Num_Students,CAPACITY
0,ALD,162,138
1,ALG,395,658
2,ARC,829,924
3,ASH,517,730
4,BAL,534,730


### Comparison of capacities to find balance

In [15]:
#schools['SCH_NUM']
#schools['CAPACITY']
imbalance = merged_df['Num_Students'] > merged_df['CAPACITY']
num_imbalance = sum(imbalance)
print('The number of imbalanced/ overpopulated schools is', num_imbalance)
balance = len(merged_df) - num_imbalance
print('The number of balanced schools is', balance)

The number of imbalanced/ overpopulated schools is 2
The number of balanced schools is 53


In [51]:
imbalance_df = merged_df[imbalance]


### Quantifying the imbalance

In [53]:
imbalance_df.loc[:,'Amount_Imbalance'] = (imbalance_df['Num_Students']-imbalance_df['CAPACITY'])

In [55]:
imbalance_df.loc[:, 'Percentage'] = imbalance_df['Amount_Imbalance']/ imbalance_df['CAPACITY']

In [56]:
imbalance_df

,ELEM_CODE,Num_Students,CAPACITY,Amount_Imbalance,Percentage
0,ALD,162,138,24,0.173913
7,BUF,1353,1039,314,0.302214


###### The percentage imbalance of 17% and 30% was recorded for ALD and BUF respectfully